This Notebook shows a few ways to visualise data on the distribution of the labour force of 162 countries, into 3 categories (Agricultural, Industry, and Service), over a span of 27 years from 1980 - 2007.

First, we import the required libraries into the notebook.

In [1]:
# -----------------------------------------------
# -----------------------------------------------
# --- HOMEWORK 1: CS-690V: DATA VISUALISATION ---
# -----------------------------------------------
# ------------- NISHIT PAREKH -------------------
# -----------------------------------------------
# -----------------------------------------------

import numpy as np
import pandas as pd
import math

from ipywidgets import interact, IntSlider

from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d, ColumnDataSource, LabelSet, Label, Legend
from bokeh.models.widgets import Slider , Select
from bokeh.io import curdoc
from bokeh.layouts import column, row, widgetbox

output_notebook()


Loading BokehJS ...

We initialise a few constants needed henceforth.

In [2]:
agro_color = (77,175,74)
indu_color = (228,26,28)
serv_color = (55,126,184)

trg_side = 100
trg_orig_x = 0
trg_orig_y = 0

We write a few helper functions. These functions are basically used by interactive elements in the notebook to generate and slice the dataset as per the user's input.

In [3]:
def makeTriangleData(data_dict,curr_year):

    year_ind_array = np.arange(num_years)
    curr_year_ind = int(year_ind_array[year_list==curr_year])

    curr_proportions = np.empty((num_countries,3),dtype=np.float32)
    curr_coordinates = np.empty((num_countries,2),dtype=np.float32)

    curr_proportions[:,0] = data_dict['agro_inter_values'][:,curr_year_ind]/100.0
    curr_proportions[:,1] = data_dict['indu_inter_values'][:,curr_year_ind]/100.0
    curr_proportions[:,2] = data_dict['serv_inter_values'][:,curr_year_ind]/100.0

    curr_coordinates[:,0] = trg_orig_x + (curr_proportions[:,1] + curr_proportions[:,2]/2.0)*trg_side
    curr_coordinates[:,1] = trg_orig_y + ((math.sqrt(3)/2.0) * curr_proportions[:,2] * trg_side)

    curr_colors = [
        "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b in zip(curr_proportions[:,1]*255,curr_proportions[:,0]*255,curr_proportions[:,2]*255)
    ]

    trg_data_dict = dict(
                        x_coords=curr_coordinates[:,0], y_coords=curr_coordinates[:,1], curr_colors=curr_colors
                    )

    return trg_data_dict

In [4]:
def makeStackedData(data_dict,curr_country):

    country_ind_array = np.arange(num_countries)
    curr_country_ind = int(country_ind_array[country_list.index(curr_country)])


    data_loc = np.invert(np.isnan(data_dict['agro_orig_values'][curr_country_ind,:]))
    nan_loc = np.invert(data_loc)

    data_loc_left_roll = np.roll(data_loc,-1)
    data_loc_right_roll = np.roll(data_loc,1)
    data_loc_left_roll[-1] = False
    data_loc_right_roll[0] = False

    left_boundary_data_loc = np.logical_and(data_loc , np.invert(data_loc_right_roll))
    right_boundary_data_loc = np.logical_and(data_loc , np.invert(data_loc_left_roll))

    year_ind_array = np.arange(num_years)

    left_boundary_data_ind = year_ind_array[left_boundary_data_loc]
    right_boundary_data_ind = year_ind_array[right_boundary_data_loc]




    base_y_values = np.zeros(num_years)

    x_labels = year_list
    agro_stacked_values = data_dict['agro_inter_values'][curr_country_ind,:]
    indu_stacked_values = data_dict['indu_inter_values'][curr_country_ind,:] + agro_stacked_values
    serv_stacked_values = data_dict['serv_inter_values'][curr_country_ind,:] + indu_stacked_values

    x_values = np.append(year_list,year_list[::-1])
    agro_y_values = np.append(agro_stacked_values,base_y_values[::-1])
    indu_y_values = np.append(indu_stacked_values,agro_stacked_values[::-1])
    serv_y_values = np.append(serv_stacked_values,indu_stacked_values[::-1])

    dashed_patch_dict = dict(
                        x_values=x_values,
                        agro_y_values=agro_y_values,
                        indu_y_values=indu_y_values,
                        serv_y_values=serv_y_values
                    )

    dashed_line_dict = dict(
                        x_labels=x_labels,
                        agro_stacked_values=agro_stacked_values,
                        indu_stacked_values=indu_stacked_values,
                        serv_stacked_values=serv_stacked_values
                    )

    patches_x_list = []
    patches_agro_y_list = []
    patches_indu_y_list = []
    patches_serv_y_list = []

    lines_x_list = []
    lines_agro_y_list = []
    lines_indu_y_list = []
    lines_serv_y_list = []

    for i in range(len(left_boundary_data_ind)):
        start_ind = left_boundary_data_ind[i]
        end_ind = right_boundary_data_ind[i]
        
        patches_x_list.append(np.append(year_list[start_ind:end_ind+1],year_list[start_ind:end_ind+1][::-1]).tolist())
        patches_agro_y_list.append(np.append(agro_stacked_values[start_ind:end_ind+1],base_y_values[start_ind:end_ind+1][::-1]).tolist())
        patches_indu_y_list.append(np.append(indu_stacked_values[start_ind:end_ind+1],agro_stacked_values[start_ind:end_ind+1][::-1]).tolist())
        patches_serv_y_list.append(np.append(serv_stacked_values[start_ind:end_ind+1],indu_stacked_values[start_ind:end_ind+1][::-1]).tolist())

        lines_x_list.append(year_list[start_ind:end_ind+1].tolist())
        lines_agro_y_list.append(agro_stacked_values[start_ind:end_ind+1].tolist())
        lines_indu_y_list.append(indu_stacked_values[start_ind:end_ind+1].tolist())
        lines_serv_y_list.append(serv_stacked_values[start_ind:end_ind+1].tolist())


    solid_patch_dict = dict(
                        patches_x_list=patches_x_list,
                        patches_agro_y_list=patches_agro_y_list,
                        patches_indu_y_list=patches_indu_y_list,
                        patches_serv_y_list=patches_serv_y_list
                    )

    solid_line_dict = dict(
                        lines_x_list=lines_x_list,
                        lines_agro_y_list=lines_agro_y_list,
                        lines_indu_y_list=lines_indu_y_list,
                        lines_serv_y_list=lines_serv_y_list
                    )


    return (dashed_patch_dict , dashed_line_dict , solid_patch_dict , solid_line_dict)

Now, we are ready to ingest the data. However, the data given is incomplete, and must be cleaned, managed and re-packaged into a suitable container to aid plotting. All that is done in the following section of code.

In [5]:
f_agro = pd.read_excel("Datasets/f_agriculture.xlsx")
f_indu = pd.read_excel("Datasets/f_industry.xlsx")
f_serv = pd.read_excel("Datasets/f_service.xlsx")
f_agro = f_agro.set_index(['Female agriculture workers (%)'])
f_indu = f_indu.set_index(['Female industry workers (%)'])
f_serv = f_serv.set_index(['Female service workers (%)'])
m_agro = pd.read_excel("Datasets/m_agriculture.xlsx")
m_indu = pd.read_excel("Datasets/m_industry.xlsx")
m_serv = pd.read_excel("Datasets/m_service.xlsx")
m_agro = m_agro.set_index(['Male agriculture workers (%)'])
m_indu = m_indu.set_index(['Male industry workers (%)'])
m_serv = m_serv.set_index(['Male service workers (%)'])

year_list = np.int32(f_agro.columns.values)
country_list = list(f_agro.index.values)
num_years = len(f_agro.columns)
num_countries = len(f_agro.index)

f_agro_orig_values = f_agro.values
f_indu_orig_values = f_indu.values
f_serv_orig_values = f_serv.values
f_agro_inter_values = f_agro.interpolate(type='linear',axis=1,limit_direction='both').values
f_indu_inter_values = f_indu.interpolate(type='linear',axis=1,limit_direction='both').values
f_serv_inter_values = f_serv.interpolate(type='linear',axis=1,limit_direction='both').values

temp_sum = f_agro_inter_values + f_indu_inter_values + f_serv_inter_values
temp_ratio = 100.0/temp_sum

f_agro_inter_values = f_agro_inter_values * temp_ratio
f_indu_inter_values = f_indu_inter_values * temp_ratio
f_serv_inter_values = f_serv_inter_values * temp_ratio


m_agro_orig_values = m_agro.values
m_indu_orig_values = m_indu.values
m_serv_orig_values = m_serv.values
m_agro_inter_values = m_agro.interpolate(type='linear',axis=1,limit_direction='both').values
m_indu_inter_values = m_indu.interpolate(type='linear',axis=1,limit_direction='both').values
m_serv_inter_values = m_serv.interpolate(type='linear',axis=1,limit_direction='both').values

temp_sum = m_agro_inter_values + m_indu_inter_values + m_serv_inter_values
temp_ratio = 100.0/temp_sum

m_agro_inter_values = m_agro_inter_values * temp_ratio
m_indu_inter_values = m_indu_inter_values * temp_ratio
m_serv_inter_values = m_serv_inter_values * temp_ratio


f_data_dict = dict(
                    agro_orig_values=     f_agro_orig_values,
                    indu_orig_values=     f_indu_orig_values,
                    serv_orig_values=     f_serv_orig_values,
                    agro_inter_values=    f_agro_inter_values,
                    indu_inter_values=    f_indu_inter_values,
                    serv_inter_values=    f_serv_inter_values
                   )


f_data = ColumnDataSource(f_data_dict)

m_data_dict = dict(
                    agro_orig_values=     m_agro_orig_values,
                    indu_orig_values=     m_indu_orig_values,
                    serv_orig_values=     m_serv_orig_values,
                    agro_inter_values=    m_agro_inter_values,
                    indu_inter_values=    m_indu_inter_values,
                    serv_inter_values=    m_serv_inter_values
                   )


m_data = ColumnDataSource(m_data_dict)

The first chart that we will use is a Triangle visualisation, that will plot each country on a triangle, depending on what the country's labour force is biased towards. Theoretically, a country with its labour force equally distributed among the 3 categories will be plotted in the centre. A country heavily invested in agriculture will be shown as a dot near the triangle's "Agriculture" corner. The colour of the dot will also indicate its bias. A <span style="color:green"> Greenish </span> shade will indicate an agricultural bias, while a <span style="color:red"> Reddish </span> shade will indicate an industrial bias, and a <span style="color:blue"> Bluish </span> shade will show a bias towards the service sector.

In [6]:
f_trg_data_dict = makeTriangleData(f_data_dict , year_list[0])
m_trg_data_dict = makeTriangleData(m_data_dict , year_list[0])

f_trg_data = ColumnDataSource(data=f_trg_data_dict)
m_trg_data = ColumnDataSource(data=m_trg_data_dict)

f_trg_fig = figure(plot_width=400,plot_height=400)
f_trg_fig.patch([trg_orig_x , trg_orig_x + (trg_side/2.0) , trg_orig_x + trg_side] , [trg_orig_y , trg_orig_y + ((math.sqrt(3)/2)*trg_side) , trg_orig_y] , line_width=2 , color='gray' , fill_alpha=0.4)
f_trg_handle = f_trg_fig.scatter('x_coords' , 'y_coords' , fill_color='curr_colors' , radius=2 , line_alpha=0, source=f_trg_data)
f_trg_fig.match_aspect = True
f_trg_fig.title.text="Labour force distribution by sector (Female)"
f_trg_fig.title.text_font_size="15px"

m_trg_fig = figure(plot_width=400,plot_height=400)
m_trg_fig.patch([trg_orig_x , trg_orig_x + (trg_side/2.0) , trg_orig_x + trg_side] , [trg_orig_y , trg_orig_y + ((math.sqrt(3)/2)*trg_side) , trg_orig_y] , line_width=2 , color='gray' , fill_alpha=0.4)
m_trg_handle = m_trg_fig.scatter('x_coords' , 'y_coords' , fill_color='curr_colors' , radius=2 , line_alpha=0, source=m_trg_data)
m_trg_fig.match_aspect = True
m_trg_fig.title.text="Labour force distribution by sector (Male)"
m_trg_fig.title.text_font_size="15px"

f_agro_label = Label(x=trg_orig_x, y=trg_orig_y,text='Agriculture', level='glyph',x_offset=-15, y_offset=5, render_mode='canvas')
f_indu_label = Label(x=trg_orig_x + (trg_side/2.0), y=trg_orig_y + ((math.sqrt(3)/2)*trg_side),text='Service', level='glyph',x_offset=-15, y_offset=5, render_mode='canvas')
f_serv_label = Label(x=trg_orig_x + trg_side, y=trg_orig_y,text='Industry', level='glyph',x_offset=-45, y_offset=5, render_mode='canvas')

m_agro_label = Label(x=trg_orig_x, y=trg_orig_y,text='Agriculture', level='glyph',x_offset=-15, y_offset=5, render_mode='canvas')
m_indu_label = Label(x=trg_orig_x + (trg_side/2.0), y=trg_orig_y + ((math.sqrt(3)/2)*trg_side),text='Service', level='glyph',x_offset=-15, y_offset=5, render_mode='canvas')
m_serv_label = Label(x=trg_orig_x + trg_side, y=trg_orig_y,text='Industry', level='glyph',x_offset=-45, y_offset=5, render_mode='canvas')

f_trg_fig.add_layout(f_agro_label)
f_trg_fig.add_layout(f_indu_label)
f_trg_fig.add_layout(f_serv_label)
m_trg_fig.add_layout(m_agro_label)
m_trg_fig.add_layout(m_indu_label)
m_trg_fig.add_layout(m_serv_label)

f_trg_fig.axis.visible = False
m_trg_fig.axis.visible = False
f_trg_fig.grid.visible = False
m_trg_fig.grid.visible = False

In [7]:
def newTriangleUpdater(year):
    
    f_new_dict = makeTriangleData(f_data_dict , year)
    f_trg_handle.data_source.data = f_new_dict

    m_new_dict = makeTriangleData(m_data_dict , year)
    m_trg_handle.data_source.data = m_new_dict
    
    push_notebook()

In [8]:
show(row(f_trg_fig,m_trg_fig),notebook_handle=True)

In [9]:
interact(newTriangleUpdater , year=IntSlider(min=year_list[0] , max=year_list[-1] , step=1 , continuous_update=True));

Use the slider to move the year, and see how the countries' bias changes.

This next chart is a Stacked Area chart, intended to show the proportion of job types in the labour force, over all the years using a single chart. Since it is a chart of proportion, it is easy to visualise the changing trends. The colour logic is the same as the previous chart.

In [10]:
f_stack_dashed_patch_data_dict , f_stack_dashed_line_data_dict , f_stack_solid_patch_data_dict , f_stack_solid_line_data_dict = makeStackedData(f_data_dict , country_list[0])
m_stack_dashed_patch_data_dict , m_stack_dashed_line_data_dict , m_stack_solid_patch_data_dict , m_stack_solid_line_data_dict = makeStackedData(m_data_dict , country_list[0])

f_stack_dashed_patch_data = ColumnDataSource(data=f_stack_dashed_patch_data_dict)
f_stack_dashed_line_data = ColumnDataSource(data=f_stack_dashed_line_data_dict)
f_stack_solid_patch_data = ColumnDataSource(data=f_stack_solid_patch_data_dict)
f_stack_solid_line_data = ColumnDataSource(data=f_stack_solid_line_data_dict)
m_stack_dashed_patch_data = ColumnDataSource(data=m_stack_dashed_patch_data_dict)
m_stack_dashed_line_data = ColumnDataSource(data=m_stack_dashed_line_data_dict)
m_stack_solid_patch_data = ColumnDataSource(data=m_stack_solid_patch_data_dict)
m_stack_solid_line_data = ColumnDataSource(data=m_stack_solid_line_data_dict)

f_stack_fig = figure(plot_width = 800, plot_height=250)
f_stack_dashed_patch_agro_handle = f_stack_fig.patch('x_values','agro_y_values',line_width=0,color=agro_color,fill_alpha=0.5,line_alpha=0,source=f_stack_dashed_patch_data)
f_stack_dashed_patch_indu_handle = f_stack_fig.patch('x_values','indu_y_values',line_width=0,color=indu_color,fill_alpha=0.5,line_alpha=0,source=f_stack_dashed_patch_data)
f_stack_dashed_patch_serv_handle = f_stack_fig.patch('x_values','serv_y_values',line_width=0,color=serv_color,fill_alpha=0.5,line_alpha=0,source=f_stack_dashed_patch_data)
f_stack_dashed_line_agro_handle = f_stack_fig.line('x_labels','agro_stacked_values',line_width=3,color=agro_color,line_dash='dashed',source=f_stack_dashed_line_data)
f_stack_dashed_line_indu_handle = f_stack_fig.line('x_labels','indu_stacked_values',line_width=3,color=indu_color,line_dash='dashed',source=f_stack_dashed_line_data)
f_stack_dashed_line_serv_handle = f_stack_fig.line('x_labels','serv_stacked_values',line_width=3,color=serv_color,line_dash='dashed',source=f_stack_dashed_line_data)
f_stack_solid_patch_agro_handle = f_stack_fig.patches('patches_x_list' , 'patches_agro_y_list' , line_width=0 , color=agro_color , source=f_stack_solid_patch_data)
f_stack_solid_patch_indu_handle = f_stack_fig.patches('patches_x_list' , 'patches_indu_y_list' , line_width=0 , color=indu_color , source=f_stack_solid_patch_data)
f_stack_solid_patch_serv_handle = f_stack_fig.patches('patches_x_list' , 'patches_serv_y_list' , line_width=0 , color=serv_color , source=f_stack_solid_patch_data)
f_stack_solid_line_agro_handle = f_stack_fig.multi_line('lines_x_list' , 'lines_agro_y_list' , line_width=3,color=agro_color , source=f_stack_solid_line_data)
f_stack_solid_line_indu_handle = f_stack_fig.multi_line('lines_x_list' , 'lines_indu_y_list' , line_width=3,color=indu_color , source=f_stack_solid_line_data)
f_stack_solid_line_serv_handle = f_stack_fig.multi_line('lines_x_list' , 'lines_serv_y_list' , line_width=3,color=serv_color , source=f_stack_solid_line_data)
f_stack_fig.x_range=Range1d(year_list[0],year_list[-1])
f_stack_fig.y_range=Range1d(0,100)
f_stack_fig.xaxis.axis_label='Year'
f_stack_fig.yaxis.axis_label='Proportion in %'
f_stack_fig.title.text="Work in Proportions (Female)"
f_stack_fig.title.text_font_size="20px"

m_stack_fig = figure(plot_width = 800, plot_height=250)
m_stack_dashed_patch_agro_handle = m_stack_fig.patch('x_values','agro_y_values',line_width=0,color=agro_color,fill_alpha=0.5,line_alpha=0,source=m_stack_dashed_patch_data)
m_stack_dashed_patch_indu_handle = m_stack_fig.patch('x_values','indu_y_values',line_width=0,color=indu_color,fill_alpha=0.5,line_alpha=0,source=m_stack_dashed_patch_data)
m_stack_dashed_patch_serv_handle = m_stack_fig.patch('x_values','serv_y_values',line_width=0,color=serv_color,fill_alpha=0.5,line_alpha=0,source=m_stack_dashed_patch_data)
m_stack_dashed_line_agro_handle = m_stack_fig.line('x_labels','agro_stacked_values',line_width=3,color=agro_color,line_dash='dashed',source=m_stack_dashed_line_data)
m_stack_dashed_line_indu_handle = m_stack_fig.line('x_labels','indu_stacked_values',line_width=3,color=indu_color,line_dash='dashed',source=m_stack_dashed_line_data)
m_stack_dashed_line_serv_handle = m_stack_fig.line('x_labels','serv_stacked_values',line_width=3,color=serv_color,line_dash='dashed',source=m_stack_dashed_line_data)
m_stack_solid_patch_agro_handle = m_stack_fig.patches('patches_x_list' , 'patches_agro_y_list' , line_width=0 , color=agro_color , source=m_stack_solid_patch_data)
m_stack_solid_patch_indu_handle = m_stack_fig.patches('patches_x_list' , 'patches_indu_y_list' , line_width=0 , color=indu_color , source=m_stack_solid_patch_data)
m_stack_solid_patch_serv_handle = m_stack_fig.patches('patches_x_list' , 'patches_serv_y_list' , line_width=0 , color=serv_color , source=m_stack_solid_patch_data)
m_stack_solid_line_agro_handle = m_stack_fig.multi_line('lines_x_list' , 'lines_agro_y_list' , line_width=3,color=agro_color , source=m_stack_solid_line_data)
m_stack_solid_line_indu_handle = m_stack_fig.multi_line('lines_x_list' , 'lines_indu_y_list' , line_width=3,color=indu_color , source=m_stack_solid_line_data)
m_stack_solid_line_serv_handle = m_stack_fig.multi_line('lines_x_list' , 'lines_serv_y_list' , line_width=3,color=serv_color , source=m_stack_solid_line_data)
m_stack_fig.x_range=Range1d(year_list[0],year_list[-1])
m_stack_fig.y_range=Range1d(0,100)
m_stack_fig.xaxis.axis_label='Year'
m_stack_fig.yaxis.axis_label='Proportion in %'
m_stack_fig.title.text="Work in Proportions (Male)"
m_stack_fig.title.text_font_size="20px"

f_stack_legend = Legend(items=[
                        ("Agriculture",[f_stack_solid_line_agro_handle]),
                        ("Industry",[f_stack_solid_line_indu_handle]),
                        ("Service",[f_stack_solid_line_serv_handle])
                    ] , location = (0,-30))

m_stack_legend = Legend(items=[
                        ("Agriculture",[m_stack_solid_line_agro_handle]),
                        ("Industry",[m_stack_solid_line_indu_handle]),
                        ("Service",[m_stack_solid_line_serv_handle])
                    ] , location = (0,-30))

f_stack_fig.add_layout(f_stack_legend,'left')
m_stack_fig.add_layout(m_stack_legend,'left')


In [11]:
def newStackUpdater(country):
    
    f_new_dashed_patch_dict , f_new_dashed_line_dict , f_new_solid_patch_dict , f_new_solid_line_dict = makeStackedData(f_data_dict , country)
    f_stack_dashed_patch_agro_handle.data_source.data = f_new_dashed_patch_dict
    f_stack_dashed_patch_serv_handle.data_source.data = f_new_dashed_patch_dict
    f_stack_dashed_patch_indu_handle.data_source.data = f_new_dashed_patch_dict
    f_stack_dashed_line_agro_handle.data_source.data = f_new_dashed_line_dict
    f_stack_dashed_line_serv_handle.data_source.data = f_new_dashed_line_dict
    f_stack_dashed_line_indu_handle.data_source.data = f_new_dashed_line_dict
    f_stack_solid_patch_agro_handle.data_source.data = f_new_solid_patch_dict
    f_stack_solid_patch_serv_handle.data_source.data = f_new_solid_patch_dict
    f_stack_solid_patch_indu_handle.data_source.data = f_new_solid_patch_dict
    f_stack_solid_line_agro_handle.data_source.data = f_new_solid_line_dict
    f_stack_solid_line_serv_handle.data_source.data = f_new_solid_line_dict
    f_stack_solid_line_indu_handle.data_source.data = f_new_solid_line_dict

    m_new_dashed_patch_dict , m_new_dashed_line_dict , m_new_solid_patch_dict , m_new_solid_line_dict = makeStackedData(m_data_dict , country)
    m_stack_dashed_patch_agro_handle.data_source.data = m_new_dashed_patch_dict
    m_stack_dashed_patch_serv_handle.data_source.data = m_new_dashed_patch_dict
    m_stack_dashed_patch_indu_handle.data_source.data = m_new_dashed_patch_dict
    m_stack_dashed_line_agro_handle.data_source.data = m_new_dashed_line_dict
    m_stack_dashed_line_serv_handle.data_source.data = m_new_dashed_line_dict
    m_stack_dashed_line_indu_handle.data_source.data = m_new_dashed_line_dict
    m_stack_solid_patch_agro_handle.data_source.data = m_new_solid_patch_dict
    m_stack_solid_patch_serv_handle.data_source.data = m_new_solid_patch_dict
    m_stack_solid_patch_indu_handle.data_source.data = m_new_solid_patch_dict
    m_stack_solid_line_agro_handle.data_source.data = m_new_solid_line_dict
    m_stack_solid_line_serv_handle.data_source.data = m_new_solid_line_dict
    m_stack_solid_line_indu_handle.data_source.data = m_new_solid_line_dict
    
    push_notebook()

In [12]:
show(column(f_stack_fig,m_stack_fig),notebook_handle=True)

In [13]:
interact(newStackUpdater , country=country_list);

The solid lines and regions indicate the existence of actual data, while the dotted lines and faded regions are the result of extrapolation for years where data was unavailable. Use the dropdown menu to select a country.